# BoxBunny Unit Tests
Quick unit tests for individual components.

## Setup Environment


In [ ]:
%%bash
cat > /tmp/boxbunny_env.sh <<'SH'
source ~/miniconda3/etc/profile.d/conda.sh
conda activate boxing_ai
export PYTHONNOUSERSITE=1
export LD_LIBRARY_PATH=$CONDA_PREFIX/lib:$LD_LIBRARY_PATH
export SETUPTOOLS_USE_DISTUTILS=stdlib
source /opt/ros/humble/setup.bash
source /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/install/setup.bash 2>/dev/null || true
SH
echo "Environment ready"

## Build Workspace

In [ ]:
!bash -lc 'source /tmp/boxbunny_env.sh && \
  cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws && \
  colcon build --symlink-install && \
  source install/setup.bash'

## 1. GUI Standalone Test
Test the main GUI layout (no ROS needed).

In [ ]:
!bash -lc 'source /tmp/boxbunny_env.sh && cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/src/boxbunny_gui/boxbunny_gui && python gui_main.py'

## 2. IMU Calibration GUI
Use this for checking IMU connection and calibrating punches.

**Workflow:**
1. Wait for IMU connection (green values in display)
2. Click 'Calibrate Strike Impact'
3. Follow prompts - hold still, then PUNCH when prompted
4. Click 'Verify File' to test detection
5. Punch the sensor - detected punches appear in the log

In [ ]:
# Build and run IMU calibration GUI
!bash -lc 'source /tmp/boxbunny_env.sh && \
  cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws && \
  colcon build --packages-select boxbunny_imu --symlink-install && \
  source install/setup.bash && \
  ros2 launch boxbunny_bringup imu_only.launch.py'

## 2b. Test Height Calibration (Service)
Trigger height calibration manually via ROS service (normally done via GUI).

In [ ]:
!bash -lc 'source /tmp/boxbunny_env.sh && \
  ros2 service call /action_predictor/calibrate_height std_srvs/srv/Trigger "{}"'

## 2c. Test Mode Switch (Service)
Switch between AI Model and Simple Color modes.

In [ ]:
# Switch to Simple Mode
!bash -lc 'source /tmp/boxbunny_env.sh && \
  ros2 service call /action_predictor/set_simple_mode std_srvs/srv/SetBool "{data: true}"'
# Switch back to AI Mode
# !bash -lc 'source /tmp/boxbunny_env.sh && \
#   ros2 service call /action_predictor/set_simple_mode std_srvs/srv/SetBool "{data: false}"'

## 3. Main System (Hybrid Mode)
Optimized Launch with YOLO Pose + IMU Fusion.

**Requirements:**
- Intel RealSense camera connected
- IMU nodes should be running (start section 2 first in another terminal)

**Modes:**
- `Hybrid Mode` (default): Uses IMU for punch detection + YOLO Pose for left/right hand
- `Anticipation Mode`: Uses full RGB-D action recognition model

In [ ]:
!bash -lc 'source /tmp/boxbunny_env.sh && \
  cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/action_prediction && \
  python tools/inference/live_infer_rgbd.py \
    --model-config configs/rgbd_boxing_anticipation.py \
    --model-checkpoint /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/models/action_prediction_model/best_acc_82.4_epoch_161.pth \
    --yolo-model ../models/checkpoints/yolo26n.pt \
    --fps 30 \
    --rgb-res 640x480 \
    --depth-res 640x480'

## 4. Parameter Tuning
Use these commands to tune detection sensitivity if needed.

In [ ]:
# Lower thresholds for more sensitive detection
!bash -lc 'source /tmp/boxbunny_env.sh && \
  ros2 param set /imu_punch_classifier accel_threshold 8.0 && \
  ros2 param set /imu_punch_classifier gyro_threshold 1.5 && \
  ros2 param set /imu_punch_classifier accel_peak_ratio 1.5 && \
  ros2 param set /imu_punch_classifier gyro_peak_ratio 1.5'

In [ ]:
# Check current parameters
!bash -lc 'source /tmp/boxbunny_env.sh && \
  ros2 param list /imu_punch_classifier'

## 5. LLM Chat GUI
Launch the LLM interface for testing.

In [ ]:
!bash -lc 'source /tmp/boxbunny_env.sh && \
  cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws && \
  colcon build --symlink-install --packages-select boxbunny_llm && \
  source install/setup.bash && \
  ros2 run boxbunny_llm llm_chat_gui'